# Section 1 - Conneting to LLM
### with an example

In [16]:
import os
from openai import OpenAI
from dotenv import load_dotenv

In [17]:
load_dotenv (dotenv_path = "C:\\Users\\Surface\\projects\\PocketMed\\.env", override = True)

True

In [18]:
openai_api_key = os.getenv ('OPENAI_API_KEY')
if openai_api_key:
    print (f"OpenAI API key exists and begins {openai_api_key [:8]}")
else:
    print ("OpenAI API key not set")

OpenAI API key exists and begins sk-proj-


In [19]:
client = OpenAI (api_key = os.environ ["OPENAI_API_KEY"])

In [20]:
question = "pick a business area that might be worth exploring for an Agentic AI opportunity. Answer in 3-4 sentences"
messages = [{"role": "user", "content": question}]
response = client.chat.completions.create (
    model = "gpt-4o",
    messages = messages,
    max_tokens = 300
)
business_area = response.choices[0].message.content.strip()
print (business_area)

A prime business area for exploring an Agentic AI opportunity is personalized mental health and wellness support. With the increasing global awareness of mental health issues, there is a growing demand for accessible, personalized, and adaptive support systems. An Agentic AI could provide real-time emotional support, recommend coping strategies, and personalize wellness plans by analyzing user interactions and biometrics. This approach not only offers a scalable solution to mental health care but also complements traditional therapy, making mental health support more accessible and timely to diverse populations.


## An agentic example

In [24]:
def ask (prompt, max_tokens = 200):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create (
        model = "gpt-4o",
        messages = messages,
        max_tokens = max_tokens,
    )
    answer = response.choices[0].message.content.strip()
    return answer

q1 = ("Q1) For a product called PocketMed — "
      "a personal health companion that tracks symptoms, meds, lifestyle, and gives personalized health insights — "
      "identify the SINGLE most valuable business area to focus on first. Answer in 1–2 short sentences. Do NOT repeat the question.")

business_area = ask (q1, max_tokens = 120)

print ("=== Q1) Business Area ===")
print (business_area)
print ("=" * 40 + "\n")


q2 = ("Q2) Based on the chosen business area below, identify ONE concrete and high-impact pain point. "
      "Then list the key PocketMed features needed to solve it — sorted based on their importance and implementation difficulty."
      "Be concise. Use bullet points only. Do NOT repeat the question.\n\n"
    f"Business area:{business_area}")

pain_point = ask (q2, max_tokens = 120)

print ("=== Q2) Pain Point ===")
print (pain_point)
print ("=" * 40 + "\n")


q3 = ("Q3) Given the business area and pain point below, propose a concise Agentic AI solution for PocketMed. Include: "
      "- the core idea (1–2 sentences)"
      "- main agents and their roles (3–5 bullets)"
      "- required data sources/tools (short list)"
      "- step-by-step rollout plan (from easiest to hardest feature to ship)"
      "Be brief and do NOT repeat the question text.\n\n"
    f"Business area:{business_area}\n\n"
    f"Pain point:{pain_point}")

solution = ask (q3, max_tokens=200)

print ("=== Q3) Agentic AI Solution ===")
print (solution)
print ("=" * 40 + "\n")

=== Q1) Business Area ===
Focus on developing and refining the personalized health insights feature to deliver highly accurate and individualized recommendations, as this will significantly enhance user engagement and trust.

=== Q2) Pain Point ===
- Pain Point: Users receive generic health recommendations that do not account for individual health data, leading to low engagement and trust.

- Key PocketMed Features Needed:
  - **Advanced Personalization Algorithm**
    - Importance: High
    - Implementation Difficulty: High

  - **User Health Profile Customization (including medical history, lifestyle, preferences)**
    - Importance: High
    - Implementation Difficulty: Medium

  - **Integration with Wearables and Health Apps for Real-Time Data Collection**
    - Importance: Medium
    - Implementation Difficulty: Medium

  - **Secure Data Handling and

=== Q3) Agentic AI Solution ===
Core Idea: Develop an advanced Agentic AI system that leverages real-time data from wearables and u

# Section 2 - Diabetes assisstant V1

In [37]:
system_prompt = """
You are PocketMed, a calm, concise diabetes information assistant and connected to an electronic health record.
- Your user is a person living with diabetes (mostly type 2) or at risk of diabetes, mostly old age.
- Use simple, clear language. Always answer in fluent Persian.
- Try to not use english medical terms and translate them in a way that an uneducated person understands (ex: use میانگین قندخون سه ماهه instead of HbA1c).
- Give short answers (2–4 sentences).
- Base your answers on standard diabetes education (lifestyle, monitoring, medications) but DO NOT give exact prescriptions, doses, or treatment orders.
- Always consider the patient's personal info if provided (age, diabetes type, medications, lab values, other conditions).
- If a situation sounds urgent (e.g. very high sugar with symptoms, chest pain, confusion, severe shortness of breath), tell them to seek immediate medical care.
"""

messages = [{"role": "system", "content": system_prompt.strip()}]

#### خلاصه و استاندارد کردن پروفایل بیمار 

In [38]:
def format_patient_profile (profile: dict) -> str: 
    parts = []
    age = profile.get ("age")
    if age is not None:
        parts.append (f"Age: {age}")
    gender = profile.get ("gender")
    if gender:
        parts.append (f"Gender: {gender}")
    dtype = profile.get ("diabetes_type")
    if dtype:
        parts.append (f"Diabetes_type: {dtype}")
    duration = profile.get ("duration_years")
    if duration is not None:
        parts.append (f"Diabetes duration (years): {duration}")
    meds = profile.get ("meds")
    if meds:
        parts.append ("Current medication: " + ", ".join (meds))
    others = profile.get ("other_conditions")
    if others:
        parts.append ("Other conditions: " + ", ".join (others))
    hba1c = profile.get ("latest_hba1c")
    if hba1c is not None:
        parts.append (f"Latest HbA1c: {hba1c}")
    if not parts:
        return "No specific patient profile was provided"
    return "Patient profile: " + " | ".join (parts)

#### لیست پیام های قبلی دستکاری نشود - پروفایل و سوال بیمار را به مدل بدهد - تاریخچه چت را آپدیت کند

In [39]:
def ask_diabetes (question: str, patient_profile: dict | None = None) -> str:
    local_messages = list (messages)
    if patient_profile:
        profile_text = format_patient_profile (patient_profile)
        local_messages.append ({"role": "user", "content": profile_text})

    local_messages.append ({"role": "user", "content": question})

    response = client.chat.completions.create (
        model = "gpt-4o",
        messages = local_messages,
        max_tokens = 300)

    answer = response.choices[0].message.content
    messages.append ({"role": "user", "content": question})
    messages.append ({"role": "assistant", "content": answer})

    return answer

#### Test

In [40]:
patientX = {
    "age": 55,
    "gender": "female",
    "diabetes_type": "type 2",
    "duration_years": 7,
    "meds": ["Metformin 1000 mg", "Gliclazide 60 mg"],
    "other_conditions": ["hypertension"],
    "latest_hba1c": 8.2
}

question = "قند من بالاست یا خوبه؟ میتونم شیرینی بخورم؟"

answer = ask_diabetes (question, patientX)
print (answer)

با توجه به میانگین قندخون سه ماهه (HbA1c) که ۸.۲ است، قند خون شما بالاتر از حد مطلوب است و باید به کنترل بهتر آن فکر کنید. در مورد مصرف شیرینی، بهتر است محدود شود و بیشتر به مصرف مواد غذایی با کربوهیدرات کم و قند طبیعی، مانند میوه‌ها و سبزیجات، توجه کنید. همچنین توصیه می‌کنم با پزشک یا کارشناس تغذیه خود مشورت کنید تا برنامه غذایی مناسبی داشته باشید.
